In [5]:
import pandas as pd

In [6]:
df=pd.read_pickle('/workspace/LLaRA/data/ref/movielens/similar_test_data.df')

In [7]:
df.head()

/opt/conda/lib/python3.10/site-packages/IPython/core/formatters.py:342: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()
/opt/conda/lib/python3.10/site-packages/IPython/core/formatters.py:342: FutureWarning: RangeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


,seq,len_seq,next,movie_names_only,seq_only,movie_embeddings,most_similar_seq_index,most_similar_seq,most_similar_seq_next,most_similar_seq_name,most_similar_seq_next_name
0,"[(704, 5), (155, 3), (80, 4), (698, 4), (745, ...",10,"(3, 4)","[House of the Spirits, The (1993), Dirty Danci...","[704, 155, 80, 698, 745, 95, 1403, 202, 731, 432]","[0.01653142, 0.06018769, 0.12848003, -0.023819...",4,"[(11, 5), (198, 5), (192, 5), (184, 5), (640, ...","(340, 3)","[Seven (Se7en) (1995), Nikita (La Femme Nikita...",Boogie Nights (1997)
1,"[(1305, 5), (135, 4), (1125, 5), (1124, 5), (2...",10,"(132, 5)","[National Lampoon's Senior Trip (1995), 2001: ...","[1305, 135, 1125, 1124, 240, 605, 477, 485, 52...","[0.002688799, -0.0696835, 0.14252476, -0.01397...",17,"[(801, 4), (529, 3), (448, 5), (569, 4), (230,...","(585, 3)","[Air Up There, The (1994), My Life as a Dog (M...",Son in Law (1993)
2,"[(1030, 5), (394, 3), (104, 3), (779, 4), (103...",10,"(671, 3)","[Beverly Hillbillies, The (1993), Radioland Mu...","[1030, 394, 104, 779, 1036, 66, 446, 1029, 451...","[-0.014814284, 0.00812782, 0.038343, 0.0050779...",29,"[(258, 3), (69, 5), (735, 5), (1021, 4), (268,...","(257, 5)","[Contact (1997), Forrest Gump (1994), Philadel...",Men in Black (1997)
3,"[(537, 4), (293, 4), (244, 3), (287, 5), (524,...",10,"(21, 5)","[My Own Private Idaho (1991), Donnie Brasco (1...","[537, 293, 244, 287, 524, 514, 922, 602, 172, 97]","[0.0041796635, -0.13311706, 0.06252247, 0.0116...",36,"[(149, 4), (509, 4), (323, 4), (461, 4), (960,...","(1524, 4)","[Jude (1996), My Left Foot (1989), Dante's Pea...",Kaspar Hauser (1993)
4,"[(11, 5), (198, 5), (192, 5), (184, 5), (640, ...",10,"(340, 3)","[Seven (Se7en) (1995), Nikita (La Femme Nikita...","[11, 198, 192, 184, 640, 581, 653, 432, 356, 666]","[-0.16389114, -0.076835975, 0.05094645, 0.1335...",50,"[(6, 3), (673, 5), (217, 4), (163, 4), (671, 3...","(86, 4)",[Shanghai Triad (Yao a yao yao dao waipo qiao)...,"Remains of the Day, The (1993)"


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   seq                         95 non-null     object
 1   len_seq                     95 non-null     int64 
 2   next                        95 non-null     object
 3   movie_names_only            95 non-null     object
 4   seq_only                    95 non-null     object
 5   movie_embeddings            95 non-null     object
 6   most_similar_seq_index      95 non-null     int64 
 7   most_similar_seq            95 non-null     object
 8   most_similar_seq_next       95 non-null     object
 9   most_similar_seq_name       95 non-null     object
 10  most_similar_seq_next_name  95 non-null     object
dtypes: int64(2), object(9)
memory usage: 8.3+ KB


In [13]:
df['most_similar_seq_name'][0]

['Seven (Se7en) (1995)',
 'Nikita (La Femme Nikita) (1990)',
 'Raging Bull (1980)',
 'Army of Darkness (1993)',
 'Cook the Thief His Wife & Her Lover, The (1989)',
 'Kalifornia (1993)',
 'Touch of Evil (1958)',
 'Fantasia (1940)',
 'Client, The (1994)',
 "Blood For Dracula (Andy Warhol's Dracula) (1974)"]

In [ ]:
import pandas as pd
import numpy as np

# 加载数据
df = pd.read_pickle('/workspace/LLaRA/data/ref/movielens/similar_test_data.df')

# 确保 seq 和 most_similar_seq 列已经被解析为列表
def safe_eval(val):
    if isinstance(val, str):
        return eval(val)
    return val

df['seq'] = df['seq'].apply(safe_eval)
df['most_similar_seq'] = df['most_similar_seq'].apply(safe_eval)

# 验证转换后的数据结构
print(df['seq'].head())
print(df['most_similar_seq'].head())

# 定义 DCG 和 nDCG 函数
def dcg_at_k(r, k):
    """计算前k个位置的DCG值"""
    r = np.asfarray(r)[:k]
    if r.size:
        return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
    return 0.0

def ndcg_at_k(r, k):
    """计算前k个位置的nDCG值"""
    dcg_max = dcg_at_k(sorted(r, reverse=True), k)
    if not dcg_max:
        return 0.0
    return dcg_at_k(r, k) / dcg_max

# 计算 nDCG@1, nDCG@2, nDCG@3
k_list = [1, 2, 3]
for k in k_list:
    df[f'ndcg@{k}'] = df.apply(lambda row: ndcg_at_k([rating for _, rating in row['most_similar_seq']], k), axis=1)

# 输出结果
print(df[['ndcg@1', 'ndcg@2', 'ndcg@3']])
